In [1]:
from mmcv import Config
from mmcv.runner import load_checkpoint
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

import wandb

apex is not installed


/opt/conda/lib/python3.7/site-packages/mmcv/utils/registry.py:250: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '


apex is not installed
apex is not installed
apex is not installed


In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
config_file_name= 'cascade_mask_rcnn_swin_small_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco'
cfg = Config.fromfile('./configs/swin/cascade_mask_rcnn_swin_small_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco.py')

PREFIX = '../../input/data/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train_all.json'
# cfg.data.train.pipe line[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
# cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
# cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4
cfg.model.neck.type = 'PAFPN'

cfg.seed=1227
cfg.gpu_ids = [0]
# cfg.work_dir = './work_dirs/cascade_swin'
cfg.work_dir = './work_dirs/!_finding_best'

cfg.runner.max_epochs = 20

# cfg.model.roi_head.bbox_head.num_classes = 11


cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [3]:
# group_name=
project_name = 'shinchan'
run_name = config_file_name
config_list = {
    'epoch': cfg.runner.max_epochs,
    'batch_size': cfg.data.samples_per_gpu,
    'optimizer': cfg.optimizer,
    'optimizer_config': cfg.optimizer_config,
    'lr_config': cfg.lr_config
}
# cfg.log_config.hooks[1].init_kwargs['group']=group_name
cfg.log_config.hooks[1].init_kwargs['name']=config_file_name
cfg.log_config.hooks[1].init_kwargs['config']=config_list
cfg.log_config.hooks[1].init_kwargs['entity'] = 'pstage3_det'

In [4]:
model = build_detector(cfg.model)
checkpoint = load_checkpoint(model, "/opt/ml/code/mmdetection_trash/work_dirs/cascade_mask_rcnn_swin_small_patch4_window7.pth", map_location='cuda:0')

2021-05-19 11:04:17,007 - mmdet - INFO - load model from: /opt/ml/code/mmdetection_trash/work_dirs/swin/swin_small_patch4_window7_224.pth
2021-05-19 11:04:17,598 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: norm.weight, norm.bias, head.weight, head.bias, layers.0.blocks.1.attn_mask, layers.1.blocks.1.attn_mask, layers.2.blocks.1.attn_mask, layers.2.blocks.3.attn_mask, layers.2.blocks.5.attn_mask, layers.2.blocks.7.attn_mask, layers.2.blocks.9.attn_mask, layers.2.blocks.11.attn_mask, layers.2.blocks.13.attn_mask, layers.2.blocks.15.attn_mask, layers.2.blocks.17.attn_mask

missing keys in source state_dict: norm0.weight, norm0.bias, norm1.weight, norm1.bias, norm2.weight, norm2.bias, norm3.weight, norm3.bias



Use load_from_local loader
The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.0.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.0.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([12]).
size mismatch for roi_head.bbox_head.0.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([44, 1024]).
size mismatch for roi_head.bbox_head.0.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([44]).
size mismatch for roi_head.bbox_head.1.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.1.fc_cls.bias: copying a param wi

In [5]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=4.41s)
creating index...
index created!


In [6]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-19 11:04:26,348 - mmdet - INFO - Start running, host: root@05a924235fca, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/!_finding_best
2021-05-19 11:04:26,348 - mmdet - INFO - workflow: [('train', 1)], max: 20 epochs


Done (t=0.75s)
creating index...
index created!


wandb: Currently logged in as: pstage3_det (use `wandb login --relogin` to force relogin)


2021-05-19 11:06:10,714 - mmdet - INFO - Epoch [1][50/818]	lr: 4.945e-06, eta: 9:04:22, time: 2.003, data_time: 0.079, memory: 17018, loss_rpn_cls: 0.4667, loss_rpn_bbox: 0.0762, s0.loss_cls: 2.3467, s0.acc: 11.0586, s0.loss_bbox: 0.3045, s0.loss_mask: 1.2604, s1.loss_cls: 1.3401, s1.acc: 0.5215, s1.loss_bbox: 0.0422, s1.loss_mask: 0.5703, s2.loss_cls: 0.6003, s2.acc: 1.7461, s2.loss_bbox: 0.0049, s2.loss_mask: 0.3232, loss: 7.3356, grad_norm: 34.0832
2021-05-19 11:07:46,820 - mmdet - INFO - Epoch [1][100/818]	lr: 9.940e-06, eta: 8:51:47, time: 1.922, data_time: 0.028, memory: 17018, loss_rpn_cls: 0.2183, loss_rpn_bbox: 0.0647, s0.loss_cls: 1.7297, s0.acc: 77.8594, s0.loss_bbox: 0.3959, s0.loss_mask: 0.9348, s1.loss_cls: 1.0148, s1.acc: 49.0771, s1.loss_bbox: 0.0561, s1.loss_mask: 0.4153, s2.loss_cls: 0.4539, s2.acc: 61.0986, s2.loss_bbox: 0.0060, s2.loss_mask: 0.2400, loss: 5.5295, grad_norm: 24.5844
2021-05-19 11:09:23,491 - mmdet - INFO - Epoch [1][150/818]	lr: 1.494e-05, eta: 8:47:

...

Loading and preparing results...
DONE (t=0.12s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.75s).
Accumulating evaluation results...


2021-05-19 21:31:56,517 - mmdet - INFO - Epoch(val) [20][818]	bbox_mAP: 0.5290, bbox_mAP_50: 0.6530, bbox_mAP_75: 0.5670, bbox_mAP_s: 0.2320, bbox_mAP_m: 0.5330, bbox_mAP_l: 0.7190, bbox_mAP_copypaste: 0.529 0.653 0.567 0.232 0.533 0.719


DONE (t=0.81s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.529
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.653
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.567
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.232
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.533
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.719
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.630
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.630
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.630
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.349
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.653
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.766


train/loss_rpn_cls,0.02686
train/loss_rpn_bbox,0.024
train/s0.loss_cls,0.16116
train/s0.acc,94.76172
train/s0.loss_bbox,0.19249
train/s0.loss_mask,0.18804
train/s1.loss_cls,0.07537
train/s1.acc,95.15385
train/s1.loss_bbox,0.07847
train/s1.loss_mask,0.08188
train/s2.loss_cls,0.03711


train/loss_rpn_cls,█▅▄▃▄▄▃▄▃▃▂▂▃▂▂▃▂▂▁▂▁▂▂▁▁▂▁▂▂▁▂▂▂▁▁▂▂▂▂▂
train/loss_rpn_bbox,█▆▅▄▄▄▄▄▄▄▄▃▃▄▃▄▃▃▂▃▂▂▃▂▂▂▃▃▂▂▂▃▂▁▃▂▄▂▂▃
train/s0.loss_cls,█▆▅▄▄▄▃▃▃▃▃▃▃▂▂▃▂▂▂▂▁▂▂▂▁▂▂▁▂▁▁▂▂▁▂▁▂▂▂▂
train/s0.acc,▁▂▃▄▄▄▅▅▅▆▆▆▅▆▆▅▇▇▇▆▇▇▇▇█▇▇▇▇▇▇▇▇█▇▇▇▇▇▇
train/s0.loss_bbox,█▅▃▃▃▃▃▃▃▂▂▂▃▂▂▃▂▂▂▂▁▂▂▂▁▂▂▂▂▁▁▂▂▁▂▂▂▂▂▁
train/s0.loss_mask,█▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
train/s1.loss_cls,▅█▆▅▅▅▄▄▄▃▃▃▃▃▂▃▂▂▂▂▁▂▂▂▁▂▂▂▂▁▁▂▂▁▂▁▂▂▂▂
train/s1.acc,▇▁▃▄▃▄▄▅▅▅▆▅▅▆▆▅▇▆▇▆▇▇▇▇█▇▇▇▇█▇▇▇█▇▇▇▇▇▇
train/s1.loss_bbox,▁█▆▅▆▅▅▅▅▄▄▄▄▄▃▅▄▄▃▄▂▃▄▃▂▃▃▃▃▃▂▃▃▂▃▃▃▃▃▂
train/s1.loss_mask,█▄▃▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/s2.loss_cls,▂█▆▅▅▅▄▄▄▄▃▃▃▃▂▄▂▂▂▂▂▂▂▂▁▂▂▂▂▁▁▂▂▁▂▁▂▂▂▂
